In [26]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import subprocess
import seaborn as sns
%matplotlib inline

In [2]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.util import makeMovie

# 位相分散解析

In [70]:
results = ['20171014-3', '20171014-6', '20171014-9', '20171014-12',
           '20171014-15', '20171014-18', '20171014-21', '20171014-24',
           '20171014-27', '20171014-30', '20171014-33', '20171014-36',
           '20171014-39', '20171014-42', '20171014-45', '20171014-48']
result_date = '20171018'

if not os.path.exists('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/case3' % result_date):
    os.makedirs('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/case3' % result_date)

In [71]:
for result in results:
    print(result)
    vmem1 = RawCam(path='/mnt/recordings/SimulationResults/%s'%result, cam_type='numpy',
                   image_width=200, image_height=200, frame_start=0, frame_end=-1)
    vmem_temp = VmemMap(vmem1)
    vmem1.cmap = vmem_temp.cmap
    vmem1.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/case3/%s/vmem'%(result_date, result))
    subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/vmem/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/vmem.mp4"'.format(result_date, result), shell=True)

    pmap = PhaseMap(vmem1, shrink=2)
    pmap.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/case3/%s/pmap'%(result_date, result))
    subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/pmap/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/pmap.mp4"'.format(result_date, result), shell=True)
    
    pvmap = PhaseVarianceMap(pmap, size=3)
    pvmap.saveImage('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/%s/case3/%s/pvmap'%(result_date, result))
    subprocess.call('ffmpeg -r 30 -y -i "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/pvmap/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/{0}/case3/{1}/pvmap.mp4"'.format(result_date, result), shell=True)

20171014-3
20171014-6
20171014-9
20171014-12
20171014-15
20171014-18
20171014-21
20171014-24
20171014-27
20171014-30
20171014-33
20171014-36
20171014-39
20171014-42
20171014-45
20171014-48


# SW消滅までの時間計算

In [19]:
"""
results = [['20171014-1', '20171014-4', '20171014-7', '20171014-10'],
           ['20171014-13', '20171014-16', '20171014-19', '20171014-22'],
           ['20171014-25', '20171014-28', '20171014-31', '20171014-34'],
           ['20171014-37', '20171014-40', '20171014-43', '20171014-46']]
"""

results =  [['20171014-16'],
           ['20171014-25', '20171014-31', '20171014-34'],
           ['20171014-40', '20171014-43', '20171014-46']]


In [20]:
for directories in results:
    for directory in directories:
        print(directory)
        vmem1 = RawCam(path='/mnt/recordings/SimulationResults/%s'%directory, cam_type='numpy',
                   image_width=200, image_height=200, frame_start=0, frame_end=-1)
        pmap = PhaseMap(vmem1, shrink=2)
        pvmap = PhaseVarianceMap(pmap, size=3)
        phase_variance =pvmap.data
        core = (phase_variance > 0.4)*1
        core_count = np.sum(np.sum(core, axis=1), axis=1)
        flag = 0
        for i in range(30, len(core_count)):
            if flag == 0 and core_count[i] == 0:
                print(i)
                flag = 1

20171014-16
187
20171014-25
188
20171014-31
186
20171014-34
186
20171014-40
186
20171014-43
186
20171014-46
186


# ヒートマップ出力

## 心筋線維走向に平行方向へ冷却した場合

In [109]:
data = [[np.nan, 7, 3, 3],
        [5, 2, 2, 2],
        [2, 2, 1, 1],
        [2, 1, 1, 1]]

In [110]:
sns.set(font_scale=1.4)
sns.heatmap(data, annot=True, fmt="1.0f", cmap='Blues_r', vmin=1, vmax=8, cbar=False)
plt.xlabel(u'Cooling width (mm)', size=16)
plt.ylabel(u'Cooling Temperature (K)', size=16)
plt.xticks([0.5,1.5,2.5,3.5],[0.15, 0.45, 0.75, 1.05], size=14)
plt.yticks([0.5,1.5,2.5,3.5],[8, 6, 4, 2], size=14)
plt.title('The number of cycles until SW annihilation', size=20)
plt.tight_layout()
plt.savefig('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171018/heatmap_parallel.png', dpi=300)
plt.close()

## 心筋線維走向と45°冷却した場合

In [111]:
data = [[np.nan, np.nan, np.nan, np.nan],
        [np.nan, np.nan, np.nan, 8],
        [np.nan, np.nan, np.nan, 2],
        [np.nan, np.nan, 2, 2]]

In [112]:
sns.set(font_scale=1.4)
sns.heatmap(data, annot=True, fmt="1.0f", cmap='Blues_r', vmin=1, vmax=8, cbar=False)
plt.xlabel(u'Cooling width (mm)', size=16)
plt.ylabel(u'Cooling Temperature (K)', size=16)
plt.xticks([0.5,1.5,2.5,3.5],[0.15, 0.45, 0.75, 1.05], size=14)
plt.yticks([0.5,1.5,2.5,3.5],[8, 6, 4, 2], size=14)
plt.title('The number of cycles until SW annihilation', size=20)
plt.tight_layout()
plt.savefig('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171018/heatmap_45.png', dpi=300)
plt.close()

## 心筋線維走向に垂直方向へ冷却した場合

In [113]:
data = [[np.nan, np.nan, np.nan, np.nan],
        [np.nan, np.nan, np.nan, np.nan],
        [np.nan, np.nan, np.nan, 3],
        [np.nan, 6, 2, 2]]

In [114]:
sns.set(font_scale=1.4)
sns.heatmap(data, annot=True, fmt="1.0f", cmap='Blues_r', vmin=1, vmax=8, cbar=False)
plt.xlabel(u'Cooling width (mm)', size=16)
plt.ylabel(u'Cooling Temperature (K)', size=16)
plt.xticks([0.5,1.5,2.5,3.5],[0.15, 0.45, 0.75, 1.05], size=14)
plt.yticks([0.5,1.5,2.5,3.5],[8, 6, 4, 2], size=14)
cbar = plt.colorbar
plt.title('The number of cycles until SW annihilation', size=20)
plt.tight_layout()
plt.savefig('/mnt/recordings/AnalysisResult/Seno/simulation/Mahajan/20171018/heatmap_vertical.png', dpi=300)
plt.close()